In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator
import pandas as pd
import seaborn as sns
from IPython.display import display
import os, sys, itertools, csv
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from mutil.gene import get_coding_genetic_target_len_d, get_intergenic_len_d
from mutil.genome import get_feature_hit_set, is_overlap, get_promoter_range_from_RegulonDB_df_row
pd.options.display.max_columns = 100

In [2]:
all_muts_df = pd.read_pickle("./data/2_df.pkl")
display(all_muts_df.shape, all_muts_df.head())

(585, 23)

,exp,ale,flask,isolate,tech_rep,presence,Position,Mutation Type,Sequence Change,Details,mutation target annotation,Reference Seq,sample,coding,range,gene RegulonDB ID,genetic features,oriC,pseudogene,TFBS,promoter,RBS,terminator
1392,GLU,3,412,2,1,1.0,13957,SNP,A→T,M599L (ATG→TTG),dnaK,NC_000913,3 412 2 1,True,"(13957, 13957)",{ECK120000235},"[{'name': 'dnaK', 'RegulonDB ID': 'ECK12000023...",False,False,{},{},{},{}
1428,GLU,3,412,2,1,1.0,28175,SNP,T→A,W295R (TGG→AGG),rihC,NC_000913,3 412 2 1,True,"(28175, 28175)",{ECK120001070},"[{'name': 'rihC', 'RegulonDB ID': 'ECK12000107...",False,False,{},{},{},{}
1393,GLU,3,412,2,1,1.0,101342,SNP,C→T,T193I (ACC→ATC),murC,NC_000913,3 412 2 1,True,"(101342, 101342)",{ECK120000612},"[{'name': 'murC', 'RegulonDB ID': 'ECK12000061...",False,False,{},{},{},{}
1394,GLU,3,412,2,1,1.0,145691,SNP,C→T,A204V (GCC→GTC),yadE,NC_000913,3 412 2 1,True,"(145691, 145691)",{ECK120001687},"[{'name': 'yadE', 'RegulonDB ID': 'ECK12000168...",False,False,{},{},{},{}
1429,GLU,3,412,2,1,1.0,171072,SNP,A→G,K166K (AAA→AAG),fhuD,NC_000913,3 412 2 1,True,"(171072, 171072)",{ECK120000299},"[{'name': 'fhuD', 'RegulonDB ID': 'ECK12000029...",False,False,{},{},{},{}


# Add component ranges
Need to find genetic features first to serve as defaults for when genomic features can't be identified.

In [3]:
gene_df = pd.read_pickle("./data/gene_df.pkl")
gene_df["range"] = gene_df.apply(lambda row: (row["GENE_POSLEFT"], row["GENE_POSRIGHT"]), axis=1)
TU_df = pd.read_pickle("./data/TU_df.pkl")
operon_df = pd.read_pickle("./data/operon_df.pkl")

gene_df.head()

,GENE_ID,GENE_NAME,GENE_POSLEFT,GENE_POSRIGHT,GENE_STRAND,GENE_SEQUENCE,GC_CONTENT,CRI_SCORE,GENE_NOTE,GENE_INTERNAL_COMMENT,KEY_ID_ORG,GENE_TYPE,range
0,ECK120000001,alr,4265782.0,4266861.0,forward,ATGCAAGCGGCAACTGTTGTGATTAACCGCCGCGCTCTGCGACACA...,55.93,NaN,NaN,NaN,ECK12,NaN,"(4265782.0, 4266861.0)"
1,ECK120000002,modB,795862.0,796551.0,forward,ATGATACTGACCGATCCAGAATGGCAGGCAGTTTTATTAAGCCTGA...,54.06,NaN,NaN,NaN,ECK12,NaN,"(795862.0, 796551.0)"
2,ECK120000003,cysZ,2531463.0,2532224.0,forward,ATGGTTTCATCATTCACATCTGCCCCACGCAGCGGTTTTTACTATT...,50.13,NaN,NaN,NaN,ECK12,NaN,"(2531463.0, 2532224.0)"
3,ECK120000004,dfp,3812731.0,3813951.0,forward,ATGAGCCTGGCCGGTAAAAAAATCGTTCTCGGCGTTAGCGGCGGTA...,53.64,NaN,NaN,NaN,ECK12,NaN,"(3812731.0, 3813951.0)"
4,ECK120000005,dcuB,4347404.0,4348744.0,reverse,ATGTTATTTACTATCCAACTTATCATAATACTGATATGTCTGTTTT...,52.27,NaN,NaN,NaN,ECK12,NaN,"(4347404.0, 4348744.0)"


In [4]:
tu_objects_df = pd.read_pickle("./data/TU_objects_df.pkl")

def get_operon_id(TU_ID, TU_df):
    operon_id = ""
    df = TU_df[TU_df["TRANSCRIPTION_UNIT_ID"]==TU_ID]
    if len(df) > 0:
        operon_id = list(df["OPERON_ID"])[0]
    return operon_id


def get_operon_name_from_operon_ID(operon_ID, operon_df):
    operon_name = "unknown"
    df = operon_df[operon_df["OPERON_ID"]==operon_ID]
    if len(df) > 0:
        operon_name = list(df["OPERON_NAME"])[0]
    return operon_name


tu_objects_df["OPERON_ID"] = tu_objects_df["TRANSCRIPTION_UNIT_ID"].apply(get_operon_id, args=[TU_df])
tu_objects_df["OPERON_NAME"] = tu_objects_df["OPERON_ID"].apply(get_operon_name_from_operon_ID, args=[operon_df])

tu_objects_df[:10]

,TRANSCRIPTION_UNIT_ID,NUMTU,TU_POSLEFT,TU_POSRIGHT,TU_TYPE,TU_OBJECT_CLASS,TU_OBJECT_ID,TU_OBJECT_NAME,TU_OBJECT_POSLEFT,TU_OBJECT_POSRIGHT,TU_OBJECT_STRAND,TU_OBJECT_COLORCLASS,TU_OBJECT_DESCRIPTION,TU_OBJECT_SIGMA,TU_OBJECT_EVIDENCE,TU_OBJECT_RI_TYPE,TU_OBJECT_TYPE,EVIDENCE,OPERON_ID,OPERON_NAME
0,ECK120008913,3,1825955,1832013,H,PM,ECK120009851,astCp1,1832013,1832013,R,NaN,NaN,Sigma70,Human inference of promoter position,NaN,predicted,NaN,ECK120014360,astCADBE
1,ECK120008913,3,1825955,1832013,H,GN,ECK120003528,astE,1825955,1826923,R,"51,153,255",amino acids,NaN,NaN,NaN,predicted,NaN,ECK120014360,astCADBE
2,ECK120008913,3,1825955,1832013,H,GN,ECK120003529,astB,1826916,1828259,R,"51,153,255",amino acids,NaN,NaN,NaN,predicted,NaN,ECK120014360,astCADBE
3,ECK120008913,3,1825955,1832013,H,GN,ECK120003532,astC,1830762,1831982,R,"0,0,255",nitrogen metabolism,NaN,NaN,NaN,predicted,NaN,ECK120014360,astCADBE
4,ECK120008913,3,1825955,1832013,H,GN,ECK120003530,astD,1828256,1829734,R,"51,153,255",amino acids,NaN,NaN,NaN,predicted,NaN,ECK120014360,astCADBE
5,ECK120008913,3,1825955,1832013,H,GN,ECK120003531,astA,1829731,1830765,R,"51,153,255",amino acids,NaN,NaN,NaN,predicted,NaN,ECK120014360,astCADBE
6,ECK120008913,3,1825955,1832013,H,PM,ECK120009851,astCp1,1832013,1832013,R,NaN,NaN,Sigma70,Human inference of promoter position,NaN,predicted,NaN,ECK120014360,astCADBE
7,ECK120008913,3,1825955,1832013,H,GN,ECK120003528,astE,1825955,1826923,R,"51,153,255",amino acids,NaN,NaN,NaN,predicted,NaN,ECK120014360,astCADBE
8,ECK120008913,3,1825955,1832013,H,GN,ECK120003529,astB,1826916,1828259,R,"51,153,255",amino acids,NaN,NaN,NaN,predicted,NaN,ECK120014360,astCADBE
9,ECK120008913,3,1825955,1832013,H,GN,ECK120003532,astC,1830762,1831982,R,"0,0,255",nitrogen metabolism,NaN,NaN,NaN,predicted,NaN,ECK120014360,astCADBE


In [5]:
# tfbs_df = pd.read_csv("./data/RegulonDBwebsite10/BindingSiteSet.txt", sep="\t", comment='#', header=None)
tfbs_df = pd.read_pickle("./data/TFBS_df.pkl")


def get_TFBS_range(tfbs_df_row):
    r = ()
    if not pd.isna(tfbs_df_row[3]) and not pd.isna(tfbs_df_row[4]):
        r = (int(tfbs_df_row[3]), int(tfbs_df_row[4])) 
    return r


tfbs_df["range"] = tfbs_df.apply(get_TFBS_range, axis=1)
display(tfbs_df.shape, tfbs_df.head())

(3562, 15)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,range
0,ECK120015994,AcrR,ECK125258528,485709,485732,reverse,ECK120033472,acrAB,-,acrAp,-22.5,gcgttagattTACATACATTTGTGAATGTATGTAccatagcacg,"[BCE|W|Binding of cellular extracts],[GEA|W|Ge...",Strong,"(485709, 485732)"
1,ECK120015994,AcrR,ECK125258528,485709,485732,forward,ECK125134945,acrR,-,acrRp,22.5,cgtgctatggTACATACATTCACAAATGTATGTAaatctaacgc,"[BCE|W|Binding of cellular extracts],[GEA|W|Ge...",Strong,"(485709, 485732)"
2,ECK120015994,AcrR,ECK125202663,1619048,1619058,forward,ECK125202664,marRAB,-,marRp,-40.5,catcggtcaaTTCATTCATTtgacttatac,"[GEA|W|Gene expression analysis],[BPP|S|Bindin...",Strong,"(1619048, 1619058)"
3,ECK120015994,AcrR,ECK125242724,1978422,1978432,reverse,ECK125242725,flhDC,-,flhDp,-31.5,tcactacacgCACATACAACggaggggggc,"[GEA|W|Gene expression analysis],[HIBSCS|W|Hum...",Weak,"(1978422, 1978432)"
4,ECK120015994,AcrR,ECK120035040,2313112,2313135,forward,ECK120035041,micF,-,micFp,41.0,atttattaccGTCATTCATTTCTGAATGTCTGTTtacccctatt,[AIBSCS|W|Automated inference based on similar...,Weak,"(2313112, 2313135)"


In [6]:
# promoter_df = pd.read_csv("./data/RegulonDB10/promoter.txt", sep="\t", comment='#', header=None)
promoter_df = pd.read_pickle("./data/promoter_df.pkl")
promoter_df["range"] = promoter_df.apply(get_promoter_range_from_RegulonDB_df_row, axis=1)
promoter_df.head()


# def get_TSS_range_from_row(promoter_df_row):
#     r = ()
#     promoter_seq_str = promoter_df_row[9]
#     if not pd.isna(promoter_seq_str):
#         TSS_genome_pos = int(promoter_df_row[3])
#         r = (TSS_genome_pos, TSS_genome_pos)
#     return r

# promoter_df["TSS range"] = promoter_df.apply(get_TSS_range_from_row, axis=1)
# promoter_df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,range
0,ECK120009842,galRp,forward,2976569.0,Sigma70,NaN,NaN,NaN,NaN,tcccacgatgaaaacacgccaccccttgaaccaacgggcgttttcc...,ECK12,NaN,NaN,"(2976509, 2976589)"
1,ECK120009843,lpxLp,reverse,1116709.0,NaN,NaN,NaN,NaN,NaN,gcggcatgatatagcaattatcgataattaacatccacacatttta...,ECK12,NaN,NaN,"(1116689, 1116769)"
2,ECK120009844,yceAp,forward,1116772.0,NaN,NaN,NaN,NaN,NaN,gcaaatgtagcgtaaaatgtgtggatgttaattatcgataattgct...,ECK12,NaN,NaN,"(1116712, 1116792)"
3,ECK120009845,mraZp,forward,89596.0,Sigma70,NaN,NaN,NaN,NaN,tatgccttgtgactggcttgacaagcttttcctcagctccgtaaac...,ECK12,The contribution of the mraZp promoter to the ...,NaN,"(89536, 89616)"
4,ECK120009846,sohBp1,forward,1329284.0,"Sigma70, Sigma38",NaN,NaN,NaN,NaN,aaatggatactttgtcatactttcgctgcaataacatctctgcgag...,ECK12,We assigned a putative transcription start sit...,NaN,"(1329224, 1329304)"


In [7]:
att_term_df = pd.read_pickle("./data/att_term_df.pkl")
att_term_df.head()

,RegulonDB ID,range
0,ECK125143526,"(200, 311)"
1,ECK125143530,"(4979, 5078)"
2,ECK125143534,"(14134, 14155)"
3,ECK125143536,"(21166, 21255)"
4,ECK125143540,"(20912, 20982)"


In [8]:
# terminator_df = pd.read_csv("./data/RegulonDB10/terminator.txt", sep="\t", comment='#', header=None)
terminator_df = pd.read_pickle("./data/term_df.pkl")
terminator_df["range"] = terminator_df.apply(lambda row: (row[2], row[3]), axis=1)
terminator_df.head()

,0,1,2,3,4,5,6,7,8,range
0,ECK120010779,NaN,2738912,2738940,rho-independent,ctgatgaaaaGGTGCCGGATGATGTGAATCATCCGGCACtggattatta,NaN,NaN,ECK12,"(2738912, 2738940)"
1,ECK120010780,NaN,2684075,2684093,rho-independent,taacgtagaaAGGCTTCCCGAAGGAAGCCttgatgatca,NaN,NaN,ECK12,"(2684075, 2684093)"
2,ECK120010781,NaN,2311610,2311624,rho-independent,caatgaaaaaAGGGCCCGCAGGCCCtttgttcgat,NaN,NaN,ECK12,"(2311610, 2311624)"
3,ECK120010782,NaN,1159325,1159346,rho-independent,tggggagactAAGGCAGCCAGATGGCTGCCTTttttacaggt,NaN,NaN,ECK12,"(1159325, 1159346)"
4,ECK120010783,NaN,1113532,1113560,rho-independent,acgagccaatAAAAATACCGGCGTTATGCCGGTATTTTTttacgaaaga,NaN,NaN,ECK12,"(1113532, 1113560)"


In [9]:
RBS_df = pd.read_pickle("./data/RBS_df.pkl")
RBS_df.head()

,SHINE_DALGARNO_ID,GENE_ID,SHINE_DALGARNO_DIST_GENE,SHINE_DALGARNO_POSLEFT,SHINE_DALGARNO_POSRIGHT,SHINE_DALGARNO_SEQUENCE,SHINE_DALGARNO_NOTE,SD_INTERNAL_COMMENT,KEY_ID_ORG,range
0,ECK120014181,ECK120000266,-11,3151252,3151257,aaattacgcgCAGGATaatatccGAT,NaN,NaN,ECK12,"(3151252, 3151257)"
1,ECK120014182,ECK120000265,-9,3151991,3151996,acttgcgtccTGGAGAtacacAGT,NaN,NaN,ECK12,"(3151991, 3151996)"
2,ECK120014183,ECK120000496,-11,3957829,3957834,acgtcaacatCGAGGGctgtcccTGT,NaN,NaN,ECK12,"(3957829, 3957834)"
3,ECK120014184,ECK120000488,-10,3957957,3957962,cacaacatcaCGAGGAatcaccATG,NaN,NaN,ECK12,"(3957957, 3957962)"
4,ECK120014185,ECK120001215,-8,3469859,3469864,tttacgtcacAAGGGAttatAAT,NaN,NaN,ECK12,"(3469859, 3469864)"


In [10]:
from mutil.mut import is_genetic_mut
all_muts_df["genetic"] = all_muts_df["Details"].apply(is_genetic_mut)
all_muts_df.head()

,exp,ale,flask,isolate,tech_rep,presence,Position,Mutation Type,Sequence Change,Details,mutation target annotation,Reference Seq,sample,coding,range,gene RegulonDB ID,genetic features,oriC,pseudogene,TFBS,promoter,RBS,terminator,genetic
1392,GLU,3,412,2,1,1.0,13957,SNP,A→T,M599L (ATG→TTG),dnaK,NC_000913,3 412 2 1,True,"(13957, 13957)",{ECK120000235},"[{'name': 'dnaK', 'RegulonDB ID': 'ECK12000023...",False,False,{},{},{},{},True
1428,GLU,3,412,2,1,1.0,28175,SNP,T→A,W295R (TGG→AGG),rihC,NC_000913,3 412 2 1,True,"(28175, 28175)",{ECK120001070},"[{'name': 'rihC', 'RegulonDB ID': 'ECK12000107...",False,False,{},{},{},{},True
1393,GLU,3,412,2,1,1.0,101342,SNP,C→T,T193I (ACC→ATC),murC,NC_000913,3 412 2 1,True,"(101342, 101342)",{ECK120000612},"[{'name': 'murC', 'RegulonDB ID': 'ECK12000061...",False,False,{},{},{},{},True
1394,GLU,3,412,2,1,1.0,145691,SNP,C→T,A204V (GCC→GTC),yadE,NC_000913,3 412 2 1,True,"(145691, 145691)",{ECK120001687},"[{'name': 'yadE', 'RegulonDB ID': 'ECK12000168...",False,False,{},{},{},{},True
1429,GLU,3,412,2,1,1.0,171072,SNP,A→G,K166K (AAA→AAG),fhuD,NC_000913,3 412 2 1,True,"(171072, 171072)",{ECK120000299},"[{'name': 'fhuD', 'RegulonDB ID': 'ECK12000029...",False,False,{},{},{},{},True


# Build component annotation

In [11]:
NONCODING_FEATURE_COLUMNS = [
    "TFBS",
    "promoter",
#     "TSS",
    "RBS",
    "terminator",
#     "attenuator terminator"
]


CODING_FEATURE_COLUMNS = ["gene RegulonDB ID"]


FEATURE_COLUMNS = NONCODING_FEATURE_COLUMNS + CODING_FEATURE_COLUMNS


def get_gene_name(RegulonDB_ID, debug=False):
    if debug:
        display(RegulonDB_ID, gene_df[gene_df["GENE_ID"] == RegulonDB_ID])
    return gene_df[gene_df["GENE_ID"] == RegulonDB_ID].iloc[0]["GENE_NAME"]  #Simply returning the first gene name in the second column that has the RegulonDB ID in the first column.


def get_feature_range(feat_id, feat_type):
    r = ()
    # default parameters
    feat_id_col = 0
    range_col = "range"
    if feat_type == "gene RegulonDB ID":
        feat_id_col = "GENE_ID"
        feat_type_df = gene_df
    elif feat_type == "TFBS":
        feat_id_col = 2
        feat_type_df = tfbs_df
    elif feat_type == "promoter":
        feat_type_df = promoter_df
#     elif feat_type == "TSS":
#         feat_type_df = promoter_df
#         range_col = "TSS range"
    elif feat_type == "RBS":
        feat_id_col = "SHINE_DALGARNO_ID"
        feat_type_df = RBS_df
    elif feat_type == "terminator":
        feat_type_df = terminator_df
    elif feat_type == "attenuator terminator":
        feat_id_col = "RegulonDB ID"
        feat_type_df = att_term_df
    
    df = feat_type_df[feat_type_df[feat_id_col] == feat_id]
    if not df.empty:
        r = df.iloc[0][range_col]  # Assuming all rows describe the same range.
    l = [int(i) for i in r]
    return tuple(l)


def _get_operon_name(TU_object_ID):
    operon_name = "unknown"
    df = tu_objects_df[tu_objects_df["TU_OBJECT_ID"] == TU_object_ID]
    if len(df) > 0:
        # non-coding features can be associated to more than one TU.
        operon_name_set = set([])
        for _, row in df.iterrows():
            operon_name_set.add(row["OPERON_NAME"])
        operon_name = ' '.join(operon_name_set)
    return operon_name


def get_genomic_feature_name(feat_id, feat_type):
    name = ""
    if feat_type == "gene RegulonDB ID":
        name = get_gene_name(feat_id)
    elif feat_type == "TFBS":  # TFBS can be assoicated to more than one TU.
        for _, row in tfbs_df[tfbs_df[2]==feat_id].iterrows():
            name += row[7] + " "
        name += "TFBS"
    elif feat_type == "promoter":
        name = promoter_df[promoter_df[0]==feat_id].iloc[0][1]
#     elif feat_type == "TSS":
#         name = promoter_df[promoter_df[0]==feat_id].iloc[0][1] + " TSS"
    elif feat_type == "RBS":
        gene_id = RBS_df[RBS_df["SHINE_DALGARNO_ID"]==feat_id].iloc[0][1]
        name = get_gene_name(gene_id) + " RBS"
    elif feat_type == "terminator":
        name = _get_operon_name(feat_id) + " " + "terminator"
    elif feat_type == "attenuator terminator":
        name = _get_operon_name(feat_id) + " " + "attenuator terminator"
    return name


def get_genomic_features(mut_row, ignore_RegDB_feats=False):
    annots = []
    if not ignore_RegDB_feats:
        for feat_col in FEATURE_COLUMNS:
            for feat_id in mut_row[feat_col]:
                d = {
                    "name": get_genomic_feature_name(feat_id, feat_col),
                    "RegulonDB ID": feat_id,
                    "range": get_feature_range(feat_id, feat_col),
                    "genetic": False if feat_col in NONCODING_FEATURE_COLUMNS else True,  # could be pseudogene which is non-coding though still genetic.
                    "feature type": "gene" if feat_col == "gene RegulonDB ID" else feat_col,
                    "operon": _get_operon_name(feat_id)
                }
                annots.append(d)
                
    if len(annots) == 0:
        annots = mut_row["genetic features"]  # Just use what's in the genetic features col.
        for a in annots:
            a["genetic"] = False if '/' in a["RegulonDB ID"] else True
            # Currently not getting operon here for integenic. Rather only getting it within the operon NB, where it's used for intergenic.
        
    return annots

all_muts_df["genomic features"] = all_muts_df.apply(lambda r: get_genomic_features(r), axis=1)
all_muts_df.head()

,exp,ale,flask,isolate,tech_rep,presence,Position,Mutation Type,Sequence Change,Details,mutation target annotation,Reference Seq,sample,coding,range,gene RegulonDB ID,genetic features,oriC,pseudogene,TFBS,promoter,RBS,terminator,genetic,genomic features
1392,GLU,3,412,2,1,1.0,13957,SNP,A→T,M599L (ATG→TTG),dnaK,NC_000913,3 412 2 1,True,"(13957, 13957)",{ECK120000235},"[{'name': 'dnaK', 'RegulonDB ID': 'ECK12000023...",False,False,{},{},{},{},True,"[{'name': 'dnaK', 'RegulonDB ID': 'ECK12000023..."
1428,GLU,3,412,2,1,1.0,28175,SNP,T→A,W295R (TGG→AGG),rihC,NC_000913,3 412 2 1,True,"(28175, 28175)",{ECK120001070},"[{'name': 'rihC', 'RegulonDB ID': 'ECK12000107...",False,False,{},{},{},{},True,"[{'name': 'rihC', 'RegulonDB ID': 'ECK12000107..."
1393,GLU,3,412,2,1,1.0,101342,SNP,C→T,T193I (ACC→ATC),murC,NC_000913,3 412 2 1,True,"(101342, 101342)",{ECK120000612},"[{'name': 'murC', 'RegulonDB ID': 'ECK12000061...",False,False,{},{},{},{},True,"[{'name': 'murC', 'RegulonDB ID': 'ECK12000061..."
1394,GLU,3,412,2,1,1.0,145691,SNP,C→T,A204V (GCC→GTC),yadE,NC_000913,3 412 2 1,True,"(145691, 145691)",{ECK120001687},"[{'name': 'yadE', 'RegulonDB ID': 'ECK12000168...",False,False,{},{},{},{},True,"[{'name': 'yadE', 'RegulonDB ID': 'ECK12000168..."
1429,GLU,3,412,2,1,1.0,171072,SNP,A→G,K166K (AAA→AAG),fhuD,NC_000913,3 412 2 1,True,"(171072, 171072)",{ECK120000299},"[{'name': 'fhuD', 'RegulonDB ID': 'ECK12000029...",False,False,{},{},{},{},True,"[{'name': 'fhuD', 'RegulonDB ID': 'ECK12000029..."


In [12]:
# Define if features are found to overlap
for _, r in all_muts_df.iterrows():
    if len(r["genomic features"]) > 1:
        for i in range(0, len(r["genomic features"])):
            feat_d1 = r["genomic features"][i]
            for j in range(i+1, len(r["genomic features"])):
                feat_d2 = r["genomic features"][j]                
                if is_overlap(feat_d1["range"], feat_d2["range"]):
                    if "overlap" not in feat_d1.keys(): feat_d1["overlap"] = set()
                    feat_d1["overlap"].add(feat_d2["name"])
                    if "overlap" not in feat_d2.keys(): feat_d2["overlap"] = set()
                    feat_d2["overlap"].add(feat_d1["name"])

In [13]:
# Remove terminators that overlap with attenuator-terminators
# The following requires all indexes to be unique
all_muts_df = all_muts_df.reset_index()

for i, r in all_muts_df.iterrows():
    if len(r["genomic features"]) > 1:
        idxs_to_remove = []
        for k in range(0, len(r["genomic features"])):
            # finding a mutated attenuator-terminator that also overlaps with a mutated terminator from the same mutation.
            feat_d = r["genomic features"][k]
            if "overlap" in feat_d.keys() \
            and feat_d["feature type"] == "terminator" \
            and feat_d["operon"]+' '+"attenuator terminator" in feat_d["overlap"]:
                idxs_to_remove.append(k)
        if len(idxs_to_remove) > 0:
            # removing the overlapping attenuator terminator
            feats = [r["genomic features"][j] for j in range(0, len(r["genomic features"])) if j not in idxs_to_remove]
            all_muts_df.at[i, "genomic features"]=feats

In [14]:
# # test code to check if any features mutated within ALEdb are completely overlapped by a gene.


# gene_df = pd.read_csv(
#     "./data/RegulonDB10/gene.txt", sep="\t", comment='#', header=None)
# gene_df.columns = [
#     "GENE_ID",
#     "GENE_NAME",
#     "GENE_POSLEFT",
#     "GENE_POSRIGHT",
#     "GENE_STRAND",
#     "GENE_SEQUENCE",
#     "GC_CONTENT",
#     "CRI_SCORE",
#     "GENE_NOTE",
#     "GENE_INTERNAL_COMMENT",
#     "KEY_ID_ORG",
#     "GENE_TYPE"
# ]
# display(len(gene_df), gene_df.head())


# for i, mut in all_muts_df.iterrows():
#     for geno_feat_d in mut["genomic features"]:
#         if not geno_feat_d["genetic"]:
#             for j, gene in gene_df.iterrows():
#                 if not pd.isna(gene.GENE_POSLEFT) and not pd.isna(gene.GENE_POSRIGHT):
#                     if (geno_feat_d["range"][0] >= gene.GENE_POSLEFT) and (geno_feat_d["range"][1] <=gene.GENE_POSRIGHT):
#                         display(geno_feat_d, gene, "##########################################")

In [15]:
all_muts_df.to_pickle("./data/2_2_df.pkl")